[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/hw-oh/wandb_e2e_demo/blob/main/models/image_segmentation/image_segmentation.ipynb)

In [1]:
!pip install -q wandb torchvision wandb-workspaces

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.3/96.3 kB 3.2 MB/s eta 0:00:00


In [3]:
import wandb
import os
from google.colab import userdata

# === Colab Secrets에서 환경변수 로드 ===
# Colab 좌측 🔑 아이콘 → 아래 키들을 등록하세요
WANDB_API_KEY = userdata.get("WANDB_API_KEY")
WANDB_ENTITY = userdata.get("WANDB_ENTITY")
WANDB_PROJECT = userdata.get("WANDB_PROJECT")
WANDB_REGISTRY_NAME = userdata.get("WANDB_REGISTRY_NAME")

os.environ["WANDB_API_KEY"] = WANDB_API_KEY

wandb.login(key=WANDB_API_KEY)
print(f"Entity: {WANDB_ENTITY}")
print(f"Project: {WANDB_PROJECT}")

/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: [wandb.login()] Using explicit session credentials for https://api.wandb.ai.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: hyunwoo-oh (wandb-korea) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Entity: wandb-korea
Project: e2e-demo-image-classification


# Image Segmentation Demo — W&B 전체 기능 체험

## 개요

이 노트북은 **Oxford-IIIT Pet Dataset**과 **DeepLabV3 (MobileNetV3-Large backbone)** 모델을 사용하여
이미지 세그멘테이션을 수행하면서, W&B(Weights & Biases)의 핵심 기능을 전부 체험합니다.

## 다루는 W&B 기능

| 기능 | 설명 |
|------|------|
| **Experiment Tracking** | 학습 메트릭 실시간 추적 (`wandb.init`, `wandb.log`, `wandb.config`) |
| **Media Logging** | 세그멘테이션 마스크 오버레이 (`wandb.Image` + `masks` 파라미터) |
| **Tables** | 데이터셋 미리보기 및 예측 결과 비교 (`wandb.Table`) |
| **Artifacts** | 데이터셋/모델 버저닝 및 계보(lineage) 추적 |
| **Model Registry** | 모델 등록 및 alias 관리 (staging) |
| **Sweeps** | 베이지안 하이퍼파라미터 최적화 |
| **Reports** | 프로그래밍 방식 실험 리포트 생성 |

## 데이터셋
- **Oxford-IIIT Pet**: ~7,400장, 37 breeds, **trimap segmentation mask** 포함
- 3-class segmentation: foreground (동물) / background (배경) / boundary (경계)
- torchvision 내장 데이터셋으로 별도 다운로드 불필요

## 모델
- **DeepLabV3** + **MobileNetV3-Large** backbone (torchvision pretrained)
- classifier head만 3-class로 교체하여 fine-tune
- ~11M 파라미터 — Colab T4에서 빠른 학습, Streamlit Cloud(CPU)에서 빠른 추론

In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as T
from torchvision.models.segmentation import deeplabv3_mobilenet_v3_large, DeepLabV3_MobileNet_V3_Large_Weights
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import numpy as np
import random
import time

CONFIG = {
    "batch_size": 16,
    "lr": 1e-3,
    "epochs": 10,
    "num_classes": 3,
    "img_size": 128,
    "model_name": "deeplabv3_mobilenetv3_large",
    "dataset": "oxford-iiit-pet",
    "optimizer": "adam",
}

SEG_CLASSES = {0: "foreground", 1: "background", 2: "boundary"}

CLASS_LABELS = {i: name for i, name in SEG_CLASSES.items()}

IMAGENET_MEAN = (0.485, 0.456, 0.406)
IMAGENET_STD = (0.229, 0.224, 0.225)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [5]:
# === Oxford-IIIT Pet 데이터 로드 + Custom Dataset ===

raw_trainval = torchvision.datasets.OxfordIIITPet(
    root="./data", split="trainval", target_types="segmentation", download=True
)
raw_test = torchvision.datasets.OxfordIIITPet(
    root="./data", split="test", target_types="segmentation", download=True
)


class PetSegDataset(Dataset):
    """Oxford-IIIT Pet segmentation 데이터셋 래퍼.

    trimap 값(1, 2, 3)을 0-indexed(0, 1, 2)로 변환하고,
    이미지와 마스크에 동일한 Resize를 적용한다.
    """

    def __init__(self, base_dataset, img_size, augment=False):
        self.base = base_dataset
        self.img_size = img_size
        self.augment = augment
        self.img_transform = T.Compose([
            T.Resize((img_size, img_size)),
            T.ToTensor(),
            T.Normalize(IMAGENET_MEAN, IMAGENET_STD),
        ])
        self.mask_resize = T.Resize(
            (img_size, img_size), interpolation=T.InterpolationMode.NEAREST
        )

    def __len__(self):
        return len(self.base)

    def __getitem__(self, idx):
        image, mask = self.base[idx]

        if self.augment and random.random() > 0.5:
            image = T.functional.hflip(image)
            mask = T.functional.hflip(mask)

        img_tensor = self.img_transform(image)
        mask_tensor = self.mask_resize(mask)
        mask_np = np.array(mask_tensor)
        # trimap: 1=foreground, 2=background, 3=boundary → 0-indexed
        mask_np = mask_np - 1
        mask_np = np.clip(mask_np, 0, 2)
        mask_tensor = torch.from_numpy(mask_np).long().squeeze(0)

        return img_tensor, mask_tensor


trainset = PetSegDataset(raw_trainval, CONFIG["img_size"], augment=True)
testset = PetSegDataset(raw_test, CONFIG["img_size"], augment=False)

trainloader = DataLoader(trainset, batch_size=CONFIG["batch_size"], shuffle=True, num_workers=2)
testloader = DataLoader(testset, batch_size=CONFIG["batch_size"], shuffle=False, num_workers=2)

print(f"Train: {len(trainset)}장, Test: {len(testset)}장")

100%|██████████| 792M/792M [00:28<00:00, 28.1MB/s]
100%|██████████| 19.2M/19.2M [00:01<00:00, 13.3MB/s]


Train: 3680장, Test: 3669장


In [6]:
# === 데이터셋 Artifact 생성 ===

run = wandb.init(
    entity=WANDB_ENTITY,
    project=WANDB_PROJECT,
    config=CONFIG,
    job_type="data-versioning",
    name="oxford-pet-data-versioning",
)

artifact = wandb.Artifact(
    "oxford-iiit-pet",
    type="dataset",
    description="Oxford-IIIT Pet dataset with trimap segmentation masks",
    metadata={
        "num_train": len(trainset),
        "num_test": len(testset),
        "num_classes": 3,
        "classes": list(SEG_CLASSES.values()),
        "image_size": f"{CONFIG['img_size']}x{CONFIG['img_size']}",
        "source": "torchvision.datasets.OxfordIIITPet",
    },
)
artifact.add_dir("./data/oxford-iiit-pet")
run.log_artifact(artifact)
print("데이터셋 Artifact 로깅 완료!")

wandb: Adding directory to artifact (data/oxford-iiit-pet)... Done. 22.7s


데이터셋 Artifact 로깅 완료!


In [7]:
# === 샘플 이미지+마스크 시각화 (wandb.Table + 마스크 오버레이) ===

def get_raw_image_and_mask(dataset, idx, img_size):
    """시각화용 원본(정규화 안 된) 이미지와 마스크를 반환한다."""
    image, mask = dataset.base[idx]
    image = image.resize((img_size, img_size))
    mask = mask.resize((img_size, img_size), Image.NEAREST)
    mask_np = np.array(mask) - 1
    mask_np = np.clip(mask_np, 0, 2)
    return image, mask_np


table = wandb.Table(columns=["Image with Mask", "Image", "Mask", "Breed"])

indices = random.sample(range(len(trainset)), 20)
for idx in indices:
    image, mask_np = get_raw_image_and_mask(trainset, idx, CONFIG["img_size"])

    img_with_mask = wandb.Image(
        image,
        masks={
            "ground_truth": {
                "mask_data": mask_np,
                "class_labels": CLASS_LABELS,
            }
        },
    )

    # 마스크를 컬러맵으로 변환 (foreground=초록, background=검정, boundary=빨강)
    color_mask = np.zeros((*mask_np.shape, 3), dtype=np.uint8)
    color_mask[mask_np == 0] = [0, 200, 0]
    color_mask[mask_np == 1] = [40, 40, 40]
    color_mask[mask_np == 2] = [200, 0, 0]

    # breed = raw_trainval._images[idx].split("_")[:-1]
    breed = raw_trainval._images[idx].stem.split("_")[:-1]
    breed_name = " ".join(breed)

    table.add_data(
        img_with_mask,
        wandb.Image(image),
        wandb.Image(Image.fromarray(color_mask)),
        breed_name,
    )

wandb.log({"dataset_preview": table})
wandb.finish()
print("데이터셋 미리보기 테이블 로깅 완료!")

AttributeError: 'PosixPath' object has no attribute 'split'

In [ ]:
# === 모델 정의 (DeepLabV3 + MobileNetV3-Large) ===

def create_model(num_classes=3):
    """DeepLabV3 + MobileNetV3-Large backbone.

    pretrained backbone 위에 classifier head만 num_classes로 교체한다.
    ~11M 파라미터로 Colab T4 학습 + Streamlit Cloud(CPU) 추론에 적합하다.
    """
    model = deeplabv3_mobilenet_v3_large(
        weights=DeepLabV3_MobileNet_V3_Large_Weights.DEFAULT
    )
    model.classifier[-1] = nn.Conv2d(256, num_classes, kernel_size=1)
    model.aux_classifier[-1] = nn.Conv2d(10, num_classes, kernel_size=1)
    return model


model = create_model(CONFIG["num_classes"]).to(device)
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Total parameters: {total_params:,}")
print(f"Trainable parameters: {trainable_params:,}")

In [ ]:
# === 학습 전 베이스라인 모델 등록 ===

run = wandb.init(
    entity=WANDB_ENTITY,
    project=WANDB_PROJECT,
    config=CONFIG,
    job_type="model-baseline",
    name="deeplabv3-pet-untrained",
)

data_artifact = run.use_artifact(f"{WANDB_ENTITY}/{WANDB_PROJECT}/oxford-iiit-pet:latest")

# 학습 전 모델 Mean IoU 측정
model.eval()
iou_sum, pixel_correct, pixel_total, num_batches = 0.0, 0, 0, 0
with torch.no_grad():
    for images, masks in testloader:
        images, masks = images.to(device), masks.to(device)
        outputs = model(images)["out"]
        preds = outputs.argmax(1)
        pixel_correct += (preds == masks).sum().item()
        pixel_total += masks.numel()
        num_batches += 1
        if num_batches >= 10:
            break

baseline_pixel_acc = pixel_correct / pixel_total
print(f"학습 전 베이스라인 Pixel Accuracy: {baseline_pixel_acc:.4f}")

wandb.log({"val/pixel_acc": baseline_pixel_acc})
wandb.summary["best_mean_iou"] = 0.0

torch.save(model.state_dict(), "untrained_model.pth")

baseline_artifact = wandb.Artifact(
    "deeplabv3-pet-seg",
    type="model",
    description="DeepLabV3 MobileNetV3-Large untrained baseline",
    metadata={
        "model_type": "segmentation",
        "model_architecture": "deeplabv3_mobilenetv3_large",
        "dataset": "oxford-iiit-pet",
        "num_classes": 3,
        "best_mean_iou": 0.0,
        "classes": list(SEG_CLASSES.values()),
        "framework": "pytorch",
        "input_size": [3, CONFIG["img_size"], CONFIG["img_size"]],
        "trained": False,
    },
)
baseline_artifact.add_file("untrained_model.pth", name="model.pth")
run.log_artifact(baseline_artifact)

run.link_artifact(
    baseline_artifact,
    f"{WANDB_REGISTRY_NAME}/pet-segmenter",
    aliases=["baseline"],
)
print("학습 전 모델을 Registry에 'baseline' alias로 등록 완료!")

wandb.finish()

In [ ]:
# === 학습 루프 ===

run = wandb.init(
    entity=WANDB_ENTITY,
    project=WANDB_PROJECT,
    config=CONFIG,
    job_type="training",
    name="deeplabv3-pet-baseline",
)

data_artifact = run.use_artifact(f"{WANDB_ENTITY}/{WANDB_PROJECT}/oxford-iiit-pet:latest")
print(f"데이터 Artifact 사용: {data_artifact.name}:{data_artifact.version}")


# --- 메트릭 계산 함수 ---

def compute_iou(pred, target, num_classes):
    """클래스별 IoU를 계산하고 평균(mIoU)을 반환한다."""
    ious = []
    for cls in range(num_classes):
        pred_cls = (pred == cls)
        target_cls = (target == cls)
        intersection = (pred_cls & target_cls).sum().item()
        union = (pred_cls | target_cls).sum().item()
        if union == 0:
            continue
        ious.append(intersection / union)
    return np.mean(ious) if ious else 0.0


def compute_pixel_accuracy(pred, target):
    return (pred == target).sum().item() / target.numel()


def compute_dice(pred, target, num_classes):
    """클래스별 Dice coefficient를 계산하고 평균을 반환한다."""
    dices = []
    for cls in range(num_classes):
        pred_cls = (pred == cls)
        target_cls = (target == cls)
        intersection = (pred_cls & target_cls).sum().item()
        total = pred_cls.sum().item() + target_cls.sum().item()
        if total == 0:
            continue
        dices.append(2.0 * intersection / total)
    return np.mean(dices) if dices else 0.0


# --- 학습/검증 함수 ---

def train_one_epoch(model, loader, criterion, optimizer):
    model.train()
    running_loss, pixel_correct, pixel_total = 0.0, 0, 0

    for batch_idx, (images, masks) in enumerate(loader):
        images, masks = images.to(device), masks.to(device)

        optimizer.zero_grad()
        outputs = model(images)["out"]
        loss = criterion(outputs, masks)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        preds = outputs.argmax(1)
        pixel_correct += (preds == masks).sum().item()
        pixel_total += masks.numel()

        if batch_idx % 50 == 0:
            wandb.log({
                "train/batch_loss": loss.item(),
            })

    return running_loss / len(loader), pixel_correct / pixel_total


def evaluate(model, loader, criterion, num_classes):
    model.eval()
    running_loss = 0.0
    total_iou, total_pixel_acc, total_dice, num_batches = 0.0, 0.0, 0.0, 0

    with torch.no_grad():
        for images, masks in loader:
            images, masks = images.to(device), masks.to(device)
            outputs = model(images)["out"]
            loss = criterion(outputs, masks)
            running_loss += loss.item()

            preds = outputs.argmax(1)
            total_iou += compute_iou(preds, masks, num_classes)
            total_pixel_acc += compute_pixel_accuracy(preds, masks)
            total_dice += compute_dice(preds, masks, num_classes)
            num_batches += 1

    n = num_batches
    return running_loss / len(loader), total_iou / n, total_pixel_acc / n, total_dice / n


# --- 학습 실행 ---

criterion = nn.CrossEntropyLoss()

if CONFIG["optimizer"] == "adam":
    optimizer = optim.Adam(model.parameters(), lr=CONFIG["lr"])
elif CONFIG["optimizer"] == "sgd":
    optimizer = optim.SGD(model.parameters(), lr=CONFIG["lr"], momentum=0.9, weight_decay=5e-4)
elif CONFIG["optimizer"] == "adamw":
    optimizer = optim.AdamW(model.parameters(), lr=CONFIG["lr"], weight_decay=1e-2)

scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=CONFIG["epochs"])

wandb.watch(model, criterion, log="all", log_freq=100)

best_iou = 0.0

for epoch in range(CONFIG["epochs"]):
    start_time = time.time()

    train_loss, train_pixel_acc = train_one_epoch(model, trainloader, criterion, optimizer)
    val_loss, val_iou, val_pixel_acc, val_dice = evaluate(
        model, testloader, criterion, CONFIG["num_classes"]
    )
    scheduler.step()
    epoch_time = time.time() - start_time

    wandb.log({
        "epoch": epoch + 1,
        "train/loss": train_loss,
        "train/pixel_acc": train_pixel_acc,
        "val/loss": val_loss,
        "val/mean_iou": val_iou,
        "val/pixel_acc": val_pixel_acc,
        "val/dice": val_dice,
        "lr": scheduler.get_last_lr()[0],
        "epoch_time_sec": epoch_time,
    })

    # 매 epoch 예측 마스크 오버레이 로깅
    model.eval()
    sample_images, sample_masks = next(iter(testloader))
    with torch.no_grad():
        sample_outputs = model(sample_images.to(device))["out"]
        sample_preds = sample_outputs.argmax(1).cpu().numpy()

    mean = torch.tensor(IMAGENET_MEAN).view(3, 1, 1)
    std = torch.tensor(IMAGENET_STD).view(3, 1, 1)

    wandb_images = []
    for i in range(min(6, len(sample_images))):
        img = sample_images[i] * std + mean
        img = img.clamp(0, 1)
        img_pil = T.ToPILImage()(img)

        wandb_images.append(wandb.Image(
            img_pil,
            masks={
                "predictions": {
                    "mask_data": sample_preds[i],
                    "class_labels": CLASS_LABELS,
                },
                "ground_truth": {
                    "mask_data": sample_masks[i].numpy(),
                    "class_labels": CLASS_LABELS,
                },
            },
        ))
    wandb.log({"predictions": wandb_images})

    if val_iou > best_iou:
        best_iou = val_iou
        torch.save(model.state_dict(), "best_model.pth")

    print(
        f"Epoch [{epoch+1}/{CONFIG['epochs']}] "
        f"Train Loss: {train_loss:.4f} | "
        f"Val Loss: {val_loss:.4f} mIoU: {val_iou:.4f} "
        f"PixAcc: {val_pixel_acc:.4f} Dice: {val_dice:.4f}"
    )

wandb.summary["best_mean_iou"] = best_iou
print(f"\n학습 완료! Best Val mIoU: {best_iou:.4f}")

In [ ]:
# === 검증 결과 wandb.Table ===

model.load_state_dict(torch.load("best_model.pth", map_location=device, weights_only=True))
model.eval()

results_table = wandb.Table(
    columns=["Image", "Ground Truth", "Prediction", "Overlay", "mIoU", "Pixel Acc", "Dice"]
)

mean = torch.tensor(IMAGENET_MEAN).view(3, 1, 1)
std = torch.tensor(IMAGENET_STD).view(3, 1, 1)

num_samples = 50
indices = random.sample(range(len(testset)), num_samples)

for idx in indices:
    img_tensor, mask_tensor = testset[idx]

    with torch.no_grad():
        output = model(img_tensor.unsqueeze(0).to(device))["out"]
        pred = output.argmax(1).squeeze(0).cpu()

    img_denorm = img_tensor * std + mean
    img_denorm = img_denorm.clamp(0, 1)
    img_pil = T.ToPILImage()(img_denorm)

    sample_iou = compute_iou(pred, mask_tensor, CONFIG["num_classes"])
    sample_pixel_acc = compute_pixel_accuracy(pred, mask_tensor)
    sample_dice = compute_dice(pred, mask_tensor, CONFIG["num_classes"])

    overlay_img = wandb.Image(
        img_pil,
        masks={
            "prediction": {
                "mask_data": pred.numpy(),
                "class_labels": CLASS_LABELS,
            },
            "ground_truth": {
                "mask_data": mask_tensor.numpy(),
                "class_labels": CLASS_LABELS,
            },
        },
    )

    # GT/Pred 마스크 컬러맵
    def mask_to_color(m):
        c = np.zeros((*m.shape, 3), dtype=np.uint8)
        c[m == 0] = [0, 200, 0]
        c[m == 1] = [40, 40, 40]
        c[m == 2] = [200, 0, 0]
        return Image.fromarray(c)

    results_table.add_data(
        wandb.Image(img_pil),
        wandb.Image(mask_to_color(mask_tensor.numpy())),
        wandb.Image(mask_to_color(pred.numpy())),
        overlay_img,
        round(sample_iou, 4),
        round(sample_pixel_acc, 4),
        round(sample_dice, 4),
    )

wandb.log({"test_predictions": results_table})
print(f"테스트 예측 결과 {num_samples}건 로깅 완료!")

In [ ]:
# === 모델 Artifact 저장 ===

model_artifact = wandb.Artifact(
    "deeplabv3-pet-seg",
    type="model",
    description="DeepLabV3 MobileNetV3-Large fine-tuned on Oxford-IIIT Pet",
    metadata={
        "model_type": "segmentation",
        "model_architecture": "deeplabv3_mobilenetv3_large",
        "dataset": "oxford-iiit-pet",
        "num_classes": 3,
        "best_mean_iou": best_iou,
        "classes": list(SEG_CLASSES.values()),
        "framework": "pytorch",
        "input_size": [3, CONFIG["img_size"], CONFIG["img_size"]],
    },
)
model_artifact.add_file("best_model.pth", name="model.pth")
run.log_artifact(model_artifact)
print("모델 Artifact 로깅 완료!")

In [ ]:
# === Model Registry 등록 ===

run.link_artifact(
    model_artifact,
    f"{WANDB_REGISTRY_NAME}/pet-segmenter",
    aliases=["staging"],
)
print("Model Registry에 'staging' alias로 등록 완료!")

wandb.finish()

## Hyperparameter Sweep

**Bayesian 최적화**를 사용하여 최적의 하이퍼파라미터 조합을 탐색합니다.

| 파라미터 | 탐색 범위 |
|-----------|------------|
| Learning Rate | 1e-5 ~ 1e-2 (log uniform) |
| Image Size | 128, 256 |
| Freeze Backbone | True, False |

In [ ]:
# === Sweep 설정 및 실행 ===

sweep_config = {
    "method": "bayes",
    "metric": {"name": "val/mean_iou", "goal": "maximize"},
    "parameters": {
        "lr": {"min": 1e-5, "max": 1e-2, "distribution": "log_uniform_values"},
        "img_size": {"values": [128, 256]},
        "freeze_backbone": {"values": [True, False]},
    },
}


def sweep_train():
    """Sweep 학습 함수"""
    run = wandb.init(entity=WANDB_ENTITY, config=CONFIG)
    config = wandb.config

    sweep_img_size = config.img_size
    sweep_trainset = PetSegDataset(raw_trainval, sweep_img_size, augment=True)
    sweep_testset = PetSegDataset(raw_test, sweep_img_size, augment=False)
    sweep_trainloader = DataLoader(
        sweep_trainset, batch_size=CONFIG["batch_size"], shuffle=True, num_workers=2
    )
    sweep_testloader = DataLoader(
        sweep_testset, batch_size=CONFIG["batch_size"], shuffle=False, num_workers=2
    )

    sweep_model = create_model(CONFIG["num_classes"]).to(device)

    if config.freeze_backbone:
        for param in sweep_model.backbone.parameters():
            param.requires_grad = False

    criterion = nn.CrossEntropyLoss()
    trainable = [p for p in sweep_model.parameters() if p.requires_grad]
    opt = optim.Adam(trainable, lr=config.lr)

    for epoch in range(3):
        train_loss, _ = train_one_epoch(sweep_model, sweep_trainloader, criterion, opt)
        val_loss, val_iou, val_pixel_acc, val_dice = evaluate(
            sweep_model, sweep_testloader, criterion, CONFIG["num_classes"]
        )
        wandb.log({
            "epoch": epoch + 1,
            "train/loss": train_loss,
            "val/loss": val_loss,
            "val/mean_iou": val_iou,
            "val/pixel_acc": val_pixel_acc,
            "val/dice": val_dice,
        })

    wandb.finish()


sweep_id = wandb.sweep(sweep_config, entity=WANDB_ENTITY, project=WANDB_PROJECT)
wandb.agent(sweep_id, function=sweep_train, count=5)
print("Sweep 완료!")

In [ ]:
# === Report 생성 ===

import wandb_workspaces.reports.v2 as wr

report = wr.Report(
    entity=WANDB_ENTITY,
    project=WANDB_PROJECT,
    title="Oxford-IIIT Pet Segmentation — 실험 결과 리포트",
    description="DeepLabV3 MobileNetV3-Large segmentation 실험 결과 및 Sweep 분석",
)

report.blocks = [
    wr.TableOfContents(),

    wr.H1("1. 실험 개요"),
    wr.P(
        "Oxford-IIIT Pet 데이터셋에 대한 DeepLabV3 이미지 세그멘테이션 실험 결과를 정리합니다. "
        "W&B의 Experiment Tracking, Artifacts, Sweeps, Model Registry 기능을 활용하였습니다."
    ),

    wr.H1("2. 학습 결과"),
    wr.PanelGrid(
        runsets=[
            wr.Runset(entity=WANDB_ENTITY, project=WANDB_PROJECT)
        ],
        panels=[
            wr.LinePlot(title="Training Loss", x="epoch", y=["train/loss"]),
            wr.LinePlot(title="Validation Loss", x="epoch", y=["val/loss"]),
            wr.LinePlot(title="Mean IoU", x="epoch", y=["val/mean_iou"]),
            wr.LinePlot(title="Pixel Accuracy", x="epoch", y=["val/pixel_acc"]),
            wr.LinePlot(title="Dice Coefficient", x="epoch", y=["val/dice"]),
        ],
    ),

    wr.H1("3. Sweep 분석"),
    wr.P("Bayesian 최적화를 통한 하이퍼파라미터 탐색 결과:"),
    wr.PanelGrid(
        runsets=[
            wr.Runset(entity=WANDB_ENTITY, project=WANDB_PROJECT)
        ],
        panels=[
            wr.ParallelCoordinatesPlot(
                columns=[
                    wr.ParallelCoordinatesPlotColumn(metric="c::lr"),
                    wr.ParallelCoordinatesPlotColumn(metric="c::img_size"),
                    wr.ParallelCoordinatesPlotColumn(metric="c::freeze_backbone"),
                    wr.ParallelCoordinatesPlotColumn(metric="val/mean_iou"),
                ],
            ),
            wr.ScalarChart(title="Best Mean IoU", metric="val/mean_iou"),
            wr.BarPlot(title="Mean IoU by Run", metrics=["val/mean_iou"]),
        ],
    ),

    wr.H1("4. 다음 단계"),
    wr.P(
        "최적 모델을 Model Registry의 'production' alias로 승격하여 "
        "Streamlit Cloud에서 실시간 추론 서비스를 제공할 수 있습니다."
    ),
]

report.save()
print(f"Report 생성 완료! URL: {report.url}")

In [ ]:
wandb.finish()
print("\n학습 & 실험 추적 완료!")
print("W&B 대시보드에서 결과를 확인하세요.")